<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 🪟 פונקציות Window ב-PySpark - מדריך מקיף
[PySpark Window Functions](https://sparkbyexamples.com/pyspark/pyspark-window-functions/)
## מבוא

פונקציות Window ב-PySpark משמשות לחישוב תוצאות כמו דירוג (rank), מספר שורה (row number) ועוד, על פני טווח של שורות קלט. פונקציות אלו שימושיות מאוד כאשר מבצעים פעולות אגרגציה במסגרת חלון (window frame) ספציפי על עמודות DataFrame.

### 🎯 מהן פונקציות Window?

פונקציות Window פועלות על קבוצה של שורות (כמו frame, partition) ומחזירות ערך בודד עבור כל שורת קלט. הן מאפשרות:
- חישוב סכומים מצטברים (running totals)
- חישוב ממוצעים נעים (moving averages)
- זיהוי ערכים קיצוניים בתוך קבוצות משנה
- דירוג ומספור שורות

### ⚡ יתרונות על פני UDF:

- **בטיחות בזמן קומפילציה** - פונקציות מובנות בטוחות יותר
- **טיפול ב-null** - מטפלות בערכי null באופן אוטומטי
- **ביצועים** - מהירות משמעותית מ-UDF מותאם אישית

---

## 📋 סוגי פונקציות Window

PySpark תומך בשלושה סוגים עיקריים של פונקציות window:

### 1️⃣ **פונקציות דירוג (Ranking Functions)**
- `row_number()` - מספר שורה סדרתי
- `rank()` - דירוג עם פערים
- `dense_rank()` - דירוג ללא פערים
- `percent_rank()` - דירוג אחוזי
- `ntile(n)` - חלוקה ל-n קבוצות

### 2️⃣ **פונקציות אנליטיות (Analytic Functions)**
- `cume_dist()` - התפלגות מצטברת
- `lag()` - גישה לשורה קודמת
- `lead()` - גישה לשורה הבאה

### 3️⃣ **פונקציות אגרגציה (Aggregate Functions)**
- `sum()`, `avg()`, `min()`, `max()`, `count()` ועוד

---

## 🔧 הגדרת הסביבה

ראשית, נייבא את הספריות הנדרושות וניצור SparkSession:

In [ ]:
import pyspark
from pyspark.sql import SparkSession

# יצירת SparkSession
spark = SparkSession.builder.appName('PySpark-Window-Functions').getOrCreate()

## 📝 יצירת DataFrame לדוגמה

ניצור DataFrame עם נתוני עובדים שישמש אותנו להדגמת פונקציות Window:

In [ ]:
# הגדרת נתונים
simpleData = (
    ("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
)

columns = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=simpleData, schema=columns)

In [ ]:
# הצגת הסכימה
df.printSchema()

In [ ]:
# הצגת הנתונים
df.show(truncate=False)

## 🪟 הגדרת Window Specification

לפני שנשתמש בפונקציות window, עלינו להגדיר את ה-window specification.

זה כולל:
- **partitionBy()** - חלוקה לקבוצות
- **orderBy()** - מיון בתוך כל קבוצה

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# הגדרת window: חלוקה לפי מחלקה, מיון לפי משכורת
windowSpec = Window.partitionBy("department").orderBy("salary")

---

# 🏆 פונקציות דירוג (Ranking Functions)

פונקציות אלו מקצות מספרים סדרתיים לשורות DataFrame על בסיס קריטריונים מוגדרים.

## 1️⃣ row_number() - מספר שורה

הפונקציה מקצה מספר שורה סדרתי ייחודי, מתחיל מ-1, לכל שורה בתוך partition:

In [ ]:
# row_number()
df.withColumn("row_number", row_number().over(windowSpec)).show(truncate=False)

## 2️⃣ rank() - דירוג עם פערים

הפונקציה מספקת דירוג לתוצאה בתוך partition. פונקציה זו משאירה פערים בדירוג כאשר יש ערכים זהים:

In [ ]:
from pyspark.sql.functions import rank

# rank()
df.withColumn("rank", rank().over(windowSpec)).show()

## 3️⃣ dense_rank() - דירוג ללא פערים

הפונקציה דומה ל-`rank()` אך ללא פערים בדירוג:

In [ ]:
from pyspark.sql.functions import dense_rank

# dense_rank()
df.withColumn("dense_rank", dense_rank().over(windowSpec)).show()

## 4️⃣ percent_rank() - דירוג אחוזי

הפונקציה מחזירה את הדירוג האחוזי של שורות בתוך partition:

In [ ]:
from pyspark.sql.functions import percent_rank

# percent_rank()
df.withColumn("percent_rank", percent_rank().over(windowSpec)).show()

## 5️⃣ ntile() - חלוקה לקבוצות

הפונקציה מחלקת את השורות ל-n קבוצות שוות (או כמעט שוות):

In [ ]:
from pyspark.sql.functions import ntile

# ntile() - חלוקה ל-2 קבוצות
df.withColumn("ntile", ntile(2).over(windowSpec)).show()

---

# 📊 פונקציות אנליטיות (Analytic Functions)

פונקציות אלו מספקות גישה לערכים משורות אחרות או מחשבות סטטיסטיקות.

## 6️⃣ cume_dist() - התפלגות מצטברת

הפונקציה מחשבת את ההתפלגות המצטברת של ערך בתוך partition:

In [ ]:
from pyspark.sql.functions import cume_dist

# cume_dist()
df.withColumn("cume_dist", cume_dist().over(windowSpec)).show()

## 7️⃣ lag() - גישה לשורה קודמת

הפונקציה מאפשרת גישה לערך מהשורה הקודמת בתוך partition:

In [ ]:
from pyspark.sql.functions import lag

# lag() - גישה לשורה שנייה לפני
df.withColumn("lag", lag("salary", 2).over(windowSpec)).show()

## 8️⃣ lead() - גישה לשורה הבאה

הפונקציה מאפשרת גישה לערך מהשורה הבאה בתוך partition:

In [ ]:
from pyspark.sql.functions import lead

# lead() - גישה לשורה שנייה אחרי
df.withColumn("lead", lead("salary", 2).over(windowSpec)).show()

---

# 📈 פונקציות אגרגציה (Aggregate Window Functions)

פונקציות אגרגציה חושבות ערכים מצטברים בתוך partitions מוגדרים.

**שים לב:** כאשר עובדים עם פונקציות אגרגציה, אין צורך ב-`orderBy` clause.

In [ ]:
from pyspark.sql.functions import col, avg, sum, min, max

# הגדרת window לאגרגציה (ללא orderBy)
windowSpecAgg = Window.partitionBy("department")

# חישוב אגרגציות לכל מחלקה
df.withColumn("row", row_number().over(windowSpec)) \
    .withColumn("avg", avg(col("salary")).over(windowSpecAgg)) \
    .withColumn("sum", sum(col("salary")).over(windowSpecAgg)) \
    .withColumn("min", min(col("salary")).over(windowSpecAgg)) \
    .withColumn("max", max(col("salary")).over(windowSpecAgg)) \
    .where(col("row") == 1).select("department", "avg", "sum", "min", "max") \
    .show()

---

## 📋 סיכום

במדריך זה למדנו על פונקציות Window השונות ב-PySpark:

### 🏆 פונקציות דירוג:
- **row_number()** - מספור ייחודי לכל שורה
- **rank()** - דירוג עם פערים בערכים זהים
- **dense_rank()** - דירוג ללא פערים
- **percent_rank()** - דירוג כאחוז (0-1)
- **ntile(n)** - חלוקה ל-n קבוצות שוות

### 📊 פונקציות אנליטיות:
- **cume_dist()** - התפלגות מצטברת (0-1)
- **lag()** - גישה לערכים משורות קודמות
- **lead()** - גישה לערכים משורות עתידיות

### 📈 פונקציות אגרגציה:
- **sum(), avg(), min(), max()** - חישובים מצטברים
- ניתן להשתמש בכל פונקציות האגרגציה הסטנדרטיות

### 💡 עקרונות חשובים:

1. **Window Specification**: תמיד הגדר `partitionBy()` למניעת העברת כל הנתונים ל-partition אחד

2. **orderBy**: נדרש לפונקציות דירוג ואנליטיות, אופציונלי לאגרגציה

3. **ביצועים**: השתמש בפונקציות Window במקום UDF - הן יעילות משמעותית

4. **מקרי שימוש נפוצים**:
   - מציאת Top N בכל קבוצה
   - חישוב ממוצעים נעים
   - השוואה בין שורות סמוכות
   - חישוב סכומים מצטברים
   - דירוג עובדים/מוצרים/לקוחות

5. **Window Frame**: ניתן להגדיר מסגרת מדויקת יותר באמצעות `rowsBetween()` או `rangeBetween()`

### 🎯 דוגמאות לשימוש:

```python
# מציאת 3 העובדים עם השכר הגבוה ביותר בכל מחלקה
df.withColumn("rank", rank().over(windowSpec)) \
  .where(col("rank") <= 3)

# חישוב הפרש משכורת מהעובד הקודם
df.withColumn("salary_diff", 
              col("salary") - lag("salary", 1).over(windowSpec))

# חישוב ממוצע נע של 3 שורות
windowFrame = Window.partitionBy("department") \
                    .orderBy("salary") \
                    .rowsBetween(-1, 1)
df.withColumn("moving_avg", avg("salary").over(windowFrame))
```

---

## 🔗 קישורים ומקורות נוספים

### מאמרים קשורים:
- [PySpark Add New Column with Row Number](https://sparkbyexamples.com/uncategorized/pyspark-add-new-column-with-row-number/)
- [Differences between rank(), dense_rank(), and row_number() functions](https://sparkbyexamples.com/pyspark/difference-between-pyspark-rank-dense_rank-and-row_number/)
- [Explain PySpark first_value() Function with Examples](https://sparkbyexamples.com/pyspark/explain-pyspark-first_value-function-with-examples/)
- [Explain PySpark last_value() Function with Examples](https://sparkbyexamples.com/pyspark/explain-pyspark-last_value-function-with-examples/)
- [Extract First and last N rows from PySpark DataFrame](https://sparkbyexamples.com/pyspark/extract-first-and-last-n-rows-from-pyspark-dataframe/)
- [PySpark Aggregate Functions with Examples](https://sparkbyexamples.com/pyspark/pyspark-aggregate-functions/)
- [PySpark JSON Functions with Examples](https://sparkbyexamples.com/pyspark/pyspark-json-functions-with-examples/)

### קוד מקור:
- [PySpark Examples GitHub - Window Functions](https://github.com/spark-examples/pyspark-examples/blob/master/pyspark-window-functions.py)

### מקור המדריך:
- [PySpark Window Functions - Spark By Examples](https://sparkbyexamples.com/pyspark/pyspark-window-functions/)

---

## 🎓 למידה מהנה!

**Happy Learning!** 🚀

*מדריך זה תורגם ונערך בעברית מתוך [SparkByExamples.com](https://sparkbyexamples.com)*